In [98]:
# data manipulations
import numpy as np
import pandas as pd

# data preaparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
# dealing with images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for writing the models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# metrics 
from sklearn.metrics import confusion_matrix, classification_report

#visualization
import matplotlib.pyplot as plt

In [107]:
def print_results(predictions, test):
    y_pred = np.argmax(predictions, axis=1)
    y_true = test.labels
    
    # confusion matrix
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))
    # classification report
    print("\nClassification report:\n")
    print(classification_report(y_true, y_pred))

"""
def graphing_training_validation():
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, ‘r’, label=’Training acc’)
    plt.plot(epochs, val_acc, ‘b’, label=’Validation acc’)
    plt.title(‘Training and validation accuracy’)
    plt.ylabel(‘accuracy’) 
    plt.xlabel(‘epoch’)
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, ‘r’, label=’Training loss’)
    plt.plot(epochs, val_loss, ‘b’, label=’Validation loss’)
    plt.title(‘Training and validation loss’)
    plt.ylabel(‘loss’) 
    plt.xlabel(‘epoch’)
    plt.legend()
    plt.show()
"""

"\ndef graphing_training_validation():\n    acc = history.history['acc']\n    val_acc = history.history['val_acc']\n    loss = history.history['loss']\n    val_loss = history.history['val_loss']\n    epochs = range(len(acc))\n    plt.plot(epochs, acc, ‘r’, label=’Training acc’)\n    plt.plot(epochs, val_acc, ‘b’, label=’Validation acc’)\n    plt.title(‘Training and validation accuracy’)\n    plt.ylabel(‘accuracy’) \n    plt.xlabel(‘epoch’)\n    plt.legend()\n    plt.figure()\n    plt.plot(epochs, loss, ‘r’, label=’Training loss’)\n    plt.plot(epochs, val_loss, ‘b’, label=’Validation loss’)\n    plt.title(‘Training and validation loss’)\n    plt.ylabel(‘loss’) \n    plt.xlabel(‘epoch’)\n    plt.legend()\n    plt.show()\n"

In [3]:
metadata = pd.read_csv("processed_metadata.csv")
metadata.head()

,path,segment,label,samplingrate,ms
0,./Labelled/Bleatings/evt_000_000_000681_210415...,0,Bleatings,16000,3000.0
1,./Labelled/Bleatings/evt_000_000_000681_210415...,1,Bleatings,16000,3000.0
2,./Labelled/Bleatings/evt_000_000_000681_210415...,2,Bleatings,16000,3000.0
3,./Labelled/Bleatings/evt_000_000_000681_210415...,3,Bleatings,16000,3000.0
4,./Labelled/Bleatings/evt_000_000_000681_210415...,4,Bleatings,16000,3000.0


# Convolutional Neural Network on log Mel-spectrogram images

In [115]:
path_dir = "./img_data"

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(128,94),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(128,94),
    batch_size=32,
    shuffle=False,
    class_mode='categorical',
    subset='validation'
)

Found 5312 images belonging to 3 classes.
Found 1327 images belonging to 3 classes.


In [110]:
validation_generator.labels

array([0, 0, 0, ..., 2, 2, 2])

In [113]:
validation_generator.classes

array([0, 0, 0, ..., 2, 2, 2])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, AveragePooling2D

## First CNN 

In [32]:
model = Sequential()
input_shape=(128, 94, 3)

#1st hidden layer
model.add(Conv2D(32, (3, 3), activation='relu', strides=(2, 2), input_shape=input_shape))
model.add(MaxPool2D((2, 2)))
#2nd hidden layer
model.add(Conv2D(64, (3, 3), activation='relu', strides=(2, 2), padding="same"))
model.add(MaxPool2D((2, 2)))
#3rd hidden layer
model.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(MaxPool2D((2, 2)))
#ANN
model.add(Flatten())
model.add(Dropout(rate=0.5))#Add fully connected layer.
model.add(Dense(units = 128, activation='relu'))
model.add(Dense(units = 64, activation='relu'))
model.add(Dropout(rate=0.25))
#Output layer
model.add(Dense(units= 3, activation='softmax'))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 63, 46, 32)        896       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 31, 23, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 16, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 8, 6, 64)          0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 8, 6, 128)         73856     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 4, 3, 128)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 1536)            

In [35]:
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [36]:
model.fit(train_generator, batch_size=32, epochs=50)

Epoch 1/50
166/166 [==============================] - 51s 305ms/step - loss: 0.6132 - accuracy: 0.7331
Epoch 2/50
166/166 [==============================] - 21s 125ms/step - loss: 0.4296 - accuracy: 0.8144
Epoch 3/50
166/166 [==============================] - 21s 126ms/step - loss: 0.3838 - accuracy: 0.8355
Epoch 4/50
166/166 [==============================] - 21s 125ms/step - loss: 0.3666 - accuracy: 0.8358
Epoch 5/50
166/166 [==============================] - 22s 130ms/step - loss: 0.3341 - accuracy: 0.8596
Epoch 6/50
166/166 [==============================] - 22s 130ms/step - loss: 0.3208 - accuracy: 0.8577
Epoch 7/50
166/166 [==============================] - 21s 128ms/step - loss: 0.3099 - accuracy: 0.8656
Epoch 8/50
166/166 [==============================] - 21s 129ms/step - loss: 0.2970 - accuracy: 0.8741
Epoch 9/50
166/166 [==============================] - 21s 129ms/step - loss: 0.2888 - accuracy: 0.8735
Epoch 10/50
166/166 [==============================] - 21s 129ms/step - l

In [74]:
test_accuracy = model.evaluate(validation_generator)

42/42 [==============================] - 5s 124ms/step - loss: 0.8039 - accuracy: 0.8772


In [93]:
prediction = model.predict(validation_generator)

In [106]:
print_results(prediction, validation_generator)

Confusion matrix:
[[300  46 222]
 [107  17  95]
 [270  54 216]]
Classification report:

              precision    recall  f1-score   support

           0       0.44      0.53      0.48       568
           1       0.15      0.08      0.10       219
           2       0.41      0.40      0.40       540

    accuracy                           0.40      1327
   macro avg       0.33      0.34      0.33      1327
weighted avg       0.38      0.40      0.39      1327



## Second CNN

In [47]:
# build a sequential model
model1 = Sequential()
model1.add(InputLayer(input_shape=(128, 94, 3)))

# 1st conv block
model1.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model1.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model1.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model1.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model1.add(BatchNormalization())
# 3rd conv block
model1.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model1.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model1.add(BatchNormalization())
# ANN block
model1.add(Flatten())
model1.add(Dense(units=100, activation='relu'))
model1.add(Dense(units=50, activation='relu'))
model1.add(Dropout(0.25))
# output layer
model1.add(Dense(units=3, activation='softmax'))
model1.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 128, 94, 25)       1900      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 64, 47, 25)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 32, 24, 50)        31300     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 16, 12, 50)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 12, 50)        200       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 6, 70)          31570     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 4, 3, 70)        

In [48]:
adam = keras.optimizers.Adam(lr=0.001)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model1.fit(train_generator, batch_size=32, epochs=50)

Epoch 1/50
166/166 [==============================] - 41s 247ms/step - loss: 0.5358 - accuracy: 0.7673
Epoch 2/50
166/166 [==============================] - 43s 261ms/step - loss: 0.3948 - accuracy: 0.8357
Epoch 3/50
166/166 [==============================] - 44s 264ms/step - loss: 0.3613 - accuracy: 0.8477
Epoch 4/50
166/166 [==============================] - 44s 266ms/step - loss: 0.3185 - accuracy: 0.8607
Epoch 5/50
166/166 [==============================] - 43s 262ms/step - loss: 0.3027 - accuracy: 0.8669
Epoch 6/50
166/166 [==============================] - 44s 267ms/step - loss: 0.2657 - accuracy: 0.8810
Epoch 7/50
166/166 [==============================] - 46s 278ms/step - loss: 0.2552 - accuracy: 0.8850
Epoch 8/50
166/166 [==============================] - 47s 283ms/step - loss: 0.2301 - accuracy: 0.9012
Epoch 9/50
166/166 [==============================] - 45s 273ms/step - loss: 0.2084 - accuracy: 0.9155
Epoch 10/50
166/166 [==============================] - 45s 273ms/step - l

In [78]:
test_accuracy = model1.evaluate(validation_generator)

42/42 [==============================] - 6s 148ms/step - loss: 1.5366 - accuracy: 0.8696


In [108]:
pred1 = model1.predict(validation_generator)
print_results(pred1, validation_generator)

Confusion matrix:
[[296  39 233]
 [105  18  96]
 [284  38 218]]

Classification report:

              precision    recall  f1-score   support

           0       0.43      0.52      0.47       568
           1       0.19      0.08      0.11       219
           2       0.40      0.40      0.40       540

    accuracy                           0.40      1327
   macro avg       0.34      0.34      0.33      1327
weighted avg       0.38      0.40      0.38      1327



## Third CNN 

In [49]:
model2 = tf.keras.models.Sequential([
    #first_convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 94, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #second_convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #third_convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #fourth_convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #out ANN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax') 
]) 
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 63, 46, 32)        896       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 31, 23, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 16, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 8, 6, 64)          0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 8, 6, 128)         73856     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 4, 3, 128)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 1536)            

In [50]:
adam = keras.optimizers.Adam(lr=0.001)
model2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model2.fit(train_generator, batch_size=32, epochs=50)

Epoch 1/50
166/166 [==============================] - 40s 243ms/step - loss: 0.5435 - accuracy: 0.7639
Epoch 2/50
166/166 [==============================] - 38s 230ms/step - loss: 0.3931 - accuracy: 0.8268
Epoch 3/50
166/166 [==============================] - 39s 235ms/step - loss: 0.3627 - accuracy: 0.8436
Epoch 4/50
166/166 [==============================] - 38s 232ms/step - loss: 0.3325 - accuracy: 0.8605
Epoch 5/50
166/166 [==============================] - 39s 237ms/step - loss: 0.3144 - accuracy: 0.8658
Epoch 6/50
166/166 [==============================] - 40s 238ms/step - loss: 0.2938 - accuracy: 0.8737
Epoch 7/50
166/166 [==============================] - 39s 234ms/step - loss: 0.2875 - accuracy: 0.8735
Epoch 8/50
166/166 [==============================] - 42s 252ms/step - loss: 0.2694 - accuracy: 0.8827
Epoch 9/50
166/166 [==============================] - 40s 240ms/step - loss: 0.2552 - accuracy: 0.8887
Epoch 10/50
166/166 [==============================] - 39s 236ms/step - l

In [117]:
test_accuracy = model2.evaluate(validation_generator)

42/42 [==============================] - 5s 107ms/step - loss: 1.0514 - accuracy: 0.8832


In [118]:
pred2 = model2.predict(validation_generator)
print_results(pred2, validation_generator)

Confusion matrix:
[[563   3   2]
 [148  71   0]
 [  2   0 538]]

Classification report:

              precision    recall  f1-score   support

           0       0.79      0.99      0.88       568
           1       0.96      0.32      0.48       219
           2       1.00      1.00      1.00       540

    accuracy                           0.88      1327
   macro avg       0.92      0.77      0.79      1327
weighted avg       0.90      0.88      0.86      1327

